In [3]:
using DataFrames, LowRankModels, CSV, LinearAlgebra, Statistics, ScikitLearn, Random, Distributions
include("proxgrad.jl")

InterruptException:
Stacktrace:
 [1] top-level scope at none:2
 [2] eval at ./boot.jl:331 [inlined]
in expression starting at /home/jacob/.julia/packages/NLSolversBase/QPnui/src/NLSolversBase.jl:5
InterruptException:
Stacktrace:
 [1] include at ./Base.jl:368 [inlined]
 [2] include(::String) at /home/jacob/.julia/packages/LowRankModels/YZgma/src/LowRankModels.jl:3
 [3] top-level scope at /home/jacob/.julia/packages/LowRankModels/YZgma/src/LowRankModels.jl:19
 [4] top-level scope at none:2
 [5] eval at ./boot.jl:331 [inlined]
in expression starting at /home/jacob/.julia/packages/LowRankModels/YZgma/src/losses.jl:39
in expression starting at /home/jacob/.julia/packages/LowRankModels/YZgma/src/LowRankModels.jl:19
┌ Info: Precompiling LowRankModels [15d4e49f-4837-5ea3-a885-5b28bfa376dc]
└ @ Base loading.jl:1278
┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1278
┌ Info: Precompiling ScikitLearn [3646fa90-6ef7-5e7e-9f22-8aca16db6324]
└ @ Base loading.jl:1

proxgrad_const

In [26]:
df = CSV.read("FinalDataSet.csv")
reddit = CSV.read("apple.csv")
twitter = df[(df[:Company] .== "AAPL"),:"Twitter Post Count"]

apple = hcat(twitter, reddit)
rename!(apple, Dict(:x1 => :"Twitter Post Count"))
rename!(apple, Dict(:"Post count" => :"Reddit Post Count"))

,Twitter Post Count,Column1,Date,Reddit Post Count,Upvote Sum,Company
,Int64,Int64,Date…,Int64,Int64,String
1,4213,0,0020-07-01,212,17198,AAPL
2,4361,1,0020-07-02,172,18516,AAPL
3,4372,2,0020-07-06,170,9617,AAPL
4,4451,3,0020-07-07,300,19666,AAPL
5,4235,4,0020-07-08,256,11261,AAPL
6,4317,5,0020-07-09,263,18116,AAPL
7,3689,6,0020-07-10,286,15810,AAPL
8,5710,7,0020-07-13,160,18535,AAPL
9,4313,8,0020-07-14,160,8528,AAPL


In [27]:
data = apple[shuffle(1:end), :]

train_proportion = 0.6
n = size(data, 1)

ntrain = convert(Int, round(train_proportion*n))

target = data[:Returns]
features = data[:, [:"Reddit Post Count", :"Twitter Post Count", :"Upvote Sum"]]

train_x = features[1:ntrain, :]
test_x = features[ntrain+1:end, :]
train_y = target[1:ntrain]
test_y = target[ntrain+1:end]

ntrain

38

In [28]:
X_train = [convert(Matrix, train_x) ones(ntrain)]
X_test = [convert(Matrix, test_x) ones(size(test_x, 1))]

ybool_train = (train_y .>= 0) 
ybool_test = (test_y .>= 0)

n = size(ybool_test, 1)

loss = 1/ntrain*LogisticLoss()
λ = 0.1
reg = λ*QuadReg()
w = proxgrad(loss, reg, X_train, ybool_train, maxiters=200000, stepsize = 0.01)

yhat = impute(loss, X_test*w)

(n - sum(yhat .== ybool_test)) / n

0.38461538461538464

In [29]:
w

4-element Array{Float64,1}:
  0.003139169901690549
  5.870991588185381e-5
 -3.4647989763126725e-5
 -6.71810662258157e-5

In [30]:
yhat

26×1 Array{Number,2}:
 false
  true
  true
  true
  true
  true
 false
  true
  true
  true
  true
  true
  true
  true
  true
  true
  true
  true
 false
  true
  true
  true
  true
  true
  true
  true